In [ ]:
import json
import pandas as pd

In [268]:
counter = 0
data_loading_bay = {"avg_borrowRate" : [], "avg_amount_borrowed" : [], "num_borrows" : [], "num_prev_defaults" : [], "default_this_time" : []}

In [261]:
import os

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1057k    0 1056k  100   176   304k     50  0:00:03  0:00:03 --:--:--  304k


In [440]:
!curl --location --request POST 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'  --data-raw '{"query":"{users(where: {borrowedReservesCount_gte: 1}, first: 1000, skip: 000) {borrowHistory { borrowRate, borrowRateMode, amount, id}, liquidationCallHistory {collateralAmount, id}}}"}' > data.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1047k    0 1047k  100   188   444k     79  0:00:02  0:00:02 --:--:--  444k


In [282]:
# counter = 0
# data_loading_bay = {"avg_borrowRate" : [], "avg_amount_borrowed" : [], "num_borrows" : [], "num_prev_defaults" : [], "default_this_time" : []}

with open("data.txt") as json_file:
  all_data = json.load(json_file)
  data = all_data["data"]
  for user in data["users"]:
    if user["borrowHistory"] != []:
      num_borrows = 0
      total_amount = 0
      total_borrowRate = 0
      num_prev_defaults = 0
      for borrow in user["borrowHistory"]:
        num_borrows += 1
        total_amount += int(borrow["amount"])
        total_borrowRate += int(borrow["borrowRate"])
        last_borrow_id = borrow["id"]
      if user["liquidationCallHistory"] != []:
        for default in user["liquidationCallHistory"]:
          num_prev_defaults = 0
          for _ in user["liquidationCallHistory"]:
            num_prev_defaults += 1
          # if default["id"] == last_borrow_id:
            default_this_time = 1
            # num_prev_defaults -= 1
          # else:
          #   default_this_time = 0
      else:
        prev_defaults = 0
        default_this_time = 0

      data_loading_bay["avg_borrowRate"].append(total_borrowRate / num_borrows)
      data_loading_bay["avg_amount_borrowed"].append(total_amount / num_borrows)
      data_loading_bay["num_borrows"].append(num_borrows)
      data_loading_bay["num_prev_defaults"].append(num_prev_defaults)
      data_loading_bay["default_this_time"].append(default_this_time)

In [314]:
nums = []
for num, thing in enumerate(data_loading_bay["default_this_time"]):
  if thing == 1:
    nums.append(num)

In [315]:
df = pd.DataFrame(data_loading_bay)  


In [316]:
for num in nums:
  data_loading_bay["avg_borrowRate"].append(df.iloc[num]["avg_borrowRate"])
  data_loading_bay["avg_amount_borrowed"].append(df.iloc[num]["avg_borrowRate"])
  data_loading_bay["num_borrows"].append(df.iloc[num]["num_borrows"])
  data_loading_bay["num_prev_defaults"].append(df.iloc[num]["num_prev_defaults"])
  data_loading_bay["default_this_time"].append(df.iloc[num]["default_this_time"])

In [318]:
df = pd.DataFrame(data_loading_bay)  

In [287]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [288]:
type(test)

pandas.core.frame.DataFrame

In [295]:
df.head(25)

,avg_borrowRate,avg_amount_borrowed,num_borrows,num_prev_defaults,default_this_time
0,4.022107e+27,3.146592e+12,5.0,0.0,0.0
1,2.988879e+25,4.415837e+23,24.0,0.0,0.0
2,6.512855e+25,8.333333e+20,12.0,0.0,0.0
3,1.745664e+25,2.000333e+21,3.0,0.0,0.0
4,4.045680e+25,2.432000e+10,5.0,0.0,0.0
5,6.499739e+25,7.818090e+08,1.0,0.0,0.0
6,8.413078e+25,5.000000e+19,3.0,0.0,0.0
7,4.797732e+25,6.406079e+21,5.0,2.0,1.0
8,3.146838e+25,3.500035e+20,5.0,0.0,0.0
9,1.588266e+26,2.322080e+21,24.0,17.0,1.0


In [319]:
df.describe()

,avg_borrowRate,avg_amount_borrowed,num_borrows,num_prev_defaults,default_this_time
count,8.457000e+03,8.457000e+03,8457.000000,8457.000000,8457.000000
mean,8.579828e+25,2.828802e+25,7.509992,1.540026,0.390682
std,2.290855e+26,5.655418e+25,14.241412,3.979588,0.487932
min,2.099765e+18,2.000000e+00,1.000000,0.000000,0.000000
25%,3.675679e+25,1.500000e+10,1.000000,0.000000,0.000000
50%,5.749018e+25,1.000000e+21,3.000000,0.000000,0.000000
75%,1.049800e+26,3.781362e+25,7.000000,1.000000,1.000000
max,1.143326e+28,6.302557e+26,100.000000,53.000000,1.000000


In [320]:
from fastai import *
from fastai.tabular.data import *

In [344]:
cont_names = df.columns

In [345]:
contt = []
for x in cont_names:
  contt.append(x)

In [346]:
contt

['avg_borrowRate',
 'avg_amount_borrowed',
 'num_borrows',
 'num_prev_defaults',
 'default_this_time']

In [367]:
contt.pop()

'default_this_time'

In [368]:
contt

['avg_borrowRate', 'avg_amount_borrowed', 'num_borrows', 'num_prev_defaults']

In [369]:
from fastai.tabular.all import *

In [350]:
# data = np.array(contt, dtype=np.float32)

In [425]:
procs = [Normalize]
# fl = FocalLoss()
# splits = RandomSplitter(valid_pct=0.2)(range_of(df))
dls = TabularDataLoaders.from_df(df = df, procs=procs, cont_names=contt, 
                                 y_names="default_this_time", 
                                 bs=64, splits = splits)
learn = tabular_learner(dls, metrics=accuracy)

In [426]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.191890,0.048161,0.608516,00:01
1,0.036027,0.041775,0.608516,00:01
2,0.012218,0.083164,0.608516,00:01
3,0.006769,0.045010,0.608516,00:01
4,0.005037,0.029467,0.608516,00:01


In [434]:
pred_data = {"avg_borrowRate" : [0], "avg_amount_borrowed" : [1], "num_borrows" : [10.0], "num_prev_defaults" : [1.0], "default_this_time" : [1.0]}
pred_df = pd.DataFrame(pred_data)
pred_df_true = pred_df

In [435]:
pred_df_true = pred_df_true.squeeze()

In [436]:
learn.predict(pred_df_true)

(   avg_borrowRate  avg_amount_borrowed  ...  num_prev_defaults  default_this_time
 0            -0.0                 -0.0  ...          -0.135104           0.977199
 
 [1 rows x 5 columns],
 tensor([0.9772]),
 tensor([0.9772]))

In [437]:
learn.export("testing.pkl")

In [441]:
!curl --location --request POST 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'  --data-raw '{"query":"{users(where: {id: "0x00d5201c4def4d1a199c63e07eac53f43252a0ea"}) {borrowHistory {borrowRate, borrowRateMode, amount, id, timestamp}, liquidationCallHistory {collateralAmount, id, timestamp}}}"}'

GraphQL server error (client error): expected `,` or `}` at line 1 column 31

In [444]:
!curl --location --request POST 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'  --data-raw '{"query":""}'

GraphQL server error (client error): expected `,` or `}` at line 1 column 31

In [470]:
os.system("""curl --location --request POST 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'  --data-raw '{"query":"{users(where: {id: {}}, first: 1000, skip: 5000) {borrowHistory { borrowRate, borrowRateMode, amount, id}, liquidationCallHistory {collateralAmount, id}}}"}'""").format("0x00d5201c4def4d1a199c63e07eac53f43252a0ea")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"errors":[{"locations":[{"line":0,"column":0}],"message":"Invalid value provided for argument `where`: Object({\"id\": Object({})})"}]}

100   302    0   136  100   166    607    741 --:--:-- --:--:-- --:--:--  1354


AttributeError: ignored

In [505]:
!curl --location --request POST 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'  --data-raw '{\"query\":\"{users(where: {id: "0x00d5201c4def4d1a199c63e07eac53f43252a0ea"}, first: 1000, skip: 5000) {borrowHistory { borrowRate, borrowRateMode, amount, id}, liquidationCallHistory {collateralAmount, id}}}\"}'

GraphQL server error (client error): key must be a string at line 1 column 2

In [500]:
os.system(cmd)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

GraphQL server error (client error): expected `,` or `}` at line 1 column 31

100   284  100    76  100   208    337    924 --:--:-- --:--:-- --:--:--  1262


0

In [506]:
!curl 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2' -H 'Accept-Encoding: gzip, deflate, br' -H 'Content-Type: application/json' -H 'Accept: application/json' -H 'Connection: keep-alive' -H 'Origin: altair://-' --data-binary '{"query":"query MyQuery {\n  users(where: {id: \"0x00d5201c4def4d1a199c63e07eac53f43252a0ea\"}) {\n    borrowHistory {\n      borrowRate\n      borrowRateMode\n      amount\n      id\n      timestamp\n    }\n    liquidationCallHistory {\n      collateralAmount\n      id\n      timestamp\n    }\n  }\n}\n","variables":{}}' --compressed

curl: (61) Unrecognized content encoding type. libcurl understands deflate, gzip content encodings.


In [514]:
!curl 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2' --data-binary '{"query":"query MyQuery {\n users(where: {id: \"0x00d5201c4def4d1a199c63e07eac53f43252a0ea\"}) {\n borrowHistory {\n borrowRate \n borrowRateMode \n amount \n id \n timestamp\n }\n liquidationCallHistory {\n collateralAmount \n id \n timestamp \n } \n} \n} \n","variables":{}}'

{"data":{"users":[{"borrowHistory":[{"amount":"2300000000000000000000","borrowRate":"203276729954722025385388386","borrowRateMode":"Variable","id":"0x2da6f59d3c184ed67fdd6409e6af958d501841bac6959f9dcefe2911cba474c8:2","timestamp":1611173341},{"amount":"2500000000","borrowRate":"292139278555399625939753728","borrowRateMode":"Variable","id":"0xb795b22647fab25892aa93251e67f15dd79d60808cd2d6621cfe19d0a5a1a20a:2","timestamp":1611514928},{"amount":"3026975203","borrowRate":"233165876824895542454053294","borrowRateMode":"Variable","id":"0xeed91838c6be0c8494f5dc74430378a0901d6d8f9e0bcc83c808863011875137:2","timestamp":1613421056}],"liquidationCallHistory":[{"collateralAmount":"148435097485258850381","id":"0x1ab7749f417bc5241110f4623f4903da7cf52371a1ba36046883be98c0cef381:8","timestamp":1621757194},{"collateralAmount":"44014590113358178953","id":"0x4bac6e47d9c5938a1afdff02800b25ba366bed39036d8f0f3da975d913fe8a4b:8","timestamp":1624340607},{"collateralAmount":"70483069086552888430","id":"0x4eb05

In [524]:
cmd = """curl 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2' --data-binary '{"query":"query MyQuery {\n users(where: {id: \"0x00d5201c4def4d1a199c63e07eac53f43252a0ea\"}) {\n borrowHistory {\n borrowRate \n borrowRateMode \n amount \n id \n timestamp\n }\n liquidationCallHistory {\n collateralAmount \n id \n timestamp \n } \n} \n} \n"}'"""

In [525]:
os.system(cmd)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

GraphQL server error (client error): control character (\u0000-\u001F) found while parsing a string at line 2 column 0

100   361  100   118  100   243    543   1119 --:--:-- --:--:-- --:--:--  1663


0

In [519]:
import subprocess

FileNotFoundError: ignored

In [559]:
graph_call = open("thing.sh", "w")
address = "0x00d5201c4def4d1a199c63e07eac53f43252a0ea"
variable = '''curl 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2' --data-binary '{"query":"query MyQuery { users(where: {id: \\"''' + address + '''\\"}) { borrowHistory { borrowRate  borrowRateMode  amount  id  timestamp } liquidationCallHistory { collateralAmount  id  timestamp  } } } ","variables":{}}' > call_data.txt'''
graph_call.write(r"{}".format(variable))
graph_call.close()

In [560]:
os.system("bash thing.sh")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1907    0  1663  100   244   6420    942 --:--:-- --:--:-- --:--:--  7362


0

In [562]:
data_loading_bay = {"avg_borrowRate" : [], "avg_amount_borrowed" : [], "num_borrows" : [], "num_prev_defaults" : [], "default_this_time" : []}

with open("call_data.txt") as json_file:
  all_data = json.load(json_file)
  data = all_data["data"]
  for user in data["users"]:
    if user["borrowHistory"] != []:
      num_borrows = 0
      total_amount = 0
      total_borrowRate = 0
      num_prev_defaults = 0
      for borrow in user["borrowHistory"]:
        num_borrows += 1
        total_amount += int(borrow["amount"])
        total_borrowRate += int(borrow["borrowRate"])
        last_borrow_id = borrow["id"]
      if user["liquidationCallHistory"] != []:
        for default in user["liquidationCallHistory"]:
          num_prev_defaults = 0
          for _ in user["liquidationCallHistory"]:
            num_prev_defaults += 1
          # if default["id"] == last_borrow_id:
            default_this_time = 1
            # num_prev_defaults -= 1
          # else:
          #   default_this_time = 0
      else:
        prev_defaults = 0
        default_this_time = 0

      data_loading_bay["avg_borrowRate"].append(total_borrowRate / num_borrows)
      data_loading_bay["avg_amount_borrowed"].append(total_amount / num_borrows)
      data_loading_bay["num_borrows"].append(num_borrows)
      data_loading_bay["num_prev_defaults"].append(num_prev_defaults)
      data_loading_bay["default_this_time"].append(default_this_time)

In [563]:
data_loading_bay

{'avg_borrowRate': [2.428606284450057e+26],
 'avg_amount_borrowed': [7.66666666668509e+20],
 'num_borrows': [3],
 'num_prev_defaults': [7],
 'default_this_time': [1]}

In [564]:
pred_df = pd.DataFrame(pred_data)
pred_df_true = pred_df_true.squeeze()
learn.predict(pred_df_true)

(   avg_borrowRate  avg_amount_borrowed  ...  num_prev_defaults  default_this_time
 0            -0.0                 -0.0  ...          -0.135104           0.977199
 
 [1 rows x 5 columns],
 tensor([0.9772]),
 tensor([0.9772]))